In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import wandb

import numpy as np
import seaborn as sns

import sys
BASE_PATH = globals()['_dh'][0].parent.absolute()
sys.path.insert(1, str(BASE_PATH))

from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import NormalizeFeatures
from src.models.iterativeModels import learnable_adaptive_iGCN

In [5]:
dataset = Planetoid(root='data/Planetoid', 
                        name="Cora", 
                        transform=NormalizeFeatures())
data = dataset[0]

Processing...
Done!


In [6]:
num_features = dataset.num_features
num_classes = dataset.num_classes
model = learnable_adaptive_iGCN(input_dim=num_features,
                                    output_dim=num_classes,
                                    hidden_dim=32,
                                    num_iterations=4,
                                    dropout=0.5)

In [15]:
print(model.train_schedule.tolist())

[0.5, 0.5, 0.5, 0.5]


In [21]:
import torch
import numpy as np
a = torch.Tensor([[1,2],[2,3]])
b = torch.Tensor(np.full(2, 1))
b

tensor([1., 1.])

In [22]:
len(b)

2

In [26]:
np.arange(0.003, 0.02, 0.0005)

array([0.003 , 0.0035, 0.004 , 0.0045, 0.005 , 0.0055, 0.006 , 0.0065,
       0.007 , 0.0075, 0.008 , 0.0085, 0.009 , 0.0095, 0.01  , 0.0105,
       0.011 , 0.0115, 0.012 , 0.0125, 0.013 , 0.0135, 0.014 , 0.0145,
       0.015 , 0.0155, 0.016 , 0.0165, 0.017 , 0.0175, 0.018 , 0.0185,
       0.019 , 0.0195])

In [14]:
b = torch.reshape(b, (, 1))
b

tensor([[1.],
        [1.]])

In [16]:
torch.cat((a,b), dim=1)

tensor([[1., 2., 1.],
        [2., 3., 1.]])

In [19]:
b.shape[0]

2

In [9]:
k = 3

tensor([3.])